In [19]:
from citations.CitationClassifier import CitationClassifier
from documents.Corpus import Corpus
from typing import List
import regex as re
from config import *
import json
import numpy as np

markdown_path = '/Users/Alex/Library/CloudStorage/GoogleDrive-alexjfogelson@gmail.com/My Drive/FutureTech/workspace/true_urop_task2_markdown/'


In [2]:

class GranularCitationClassier(CitationClassifier):
    def __init__(self):
        pass
    
    def classify_text(self, text: str, labels: List[str]) -> str:
        fine_tune = bool(re.findall(r'fine\-tun', text))
        in_methods = 'method' in labels
        
        if fine_tune and in_methods:
            return 'fine-tune'
        
        return 'not-fine-tune'
        
    
    def getClassificationOrdering(self) -> List[str]:
        return ['fine-tune', 'not-fine-tune']
    
    def getClassificationRanking(self, result):
        return self.getClassificationOrdering().index(result)

In [3]:
classifier = GranularCitationClassier()
corpus = Corpus(markdown_path, 
                    extensions = ['mmd'])

with open(FOUNDATION_MODELS_PATH, 'r') as f:
    foundational_models_json = json.load(f)
    keys_, titles_, ids_ = list(zip(*[(key, data['title'].replace('\\infty', '∞'), data['paperId']) for key, data in foundational_models_json.items()]))
    keys_, titles_, ids_ = list(keys_), list(titles_), list(ids_)

foundation_model_filter = [True for id in ids_]
keys, titles = np.array(keys_)[foundation_model_filter].tolist(), np.array(titles_)[foundation_model_filter].tolist()

corpus.findAllPaperRefsAllTitles(titles = titles, keys = keys, classifier = classifier)

textualRefs = corpus.getAllTextualReferences(as_dict=True)

100%|██████████| 953/953 [00:40<00:00, 23.77it/s]


In [11]:
fine_tune = [ref for ref in textualRefs if ref['classification'] == 'fine-tune']

In [18]:
from random import randint

fine_tune[randint(0, len(fine_tune))]

{'sentence': '#### 5.6.1 fine-tuning datasets\n\npangu-coder [33] introduces additional datasets which contain apps [66] and code contests <cite>[67]</cite> (cc) datasets from a more similar distribution.',
 'classification': 'fine-tune',
 'classification_order': 0,
 'labels': {'method'},
 'FM_key': '193_alphacode',
 'FM_title': 'competition-level code generation with alphacode',
 'paperId': '362cbfd0d05e139cd6cf049754098a6e1520b910.mmd'}